In [28]:
import json

import networkx as nx

In [4]:
ls data

emp-posts.json  ent-terms.json  per-posts.json  pub-terms.json
emp-terms.json  exp-posts.json  per-terms.json  ter-posts.json
ent-posts.json  exp-terms.json  pub-posts.json  ter-terms.json


In [14]:
# Mappings

classes = {
    'entity': 'Entidad',
    'person': 'Persona',
    'book': 'Publicación',
    'company': 'Empresa',
    'exhibition': 'Exposición',
}

In [32]:
graph = nx.DiGraph()

In [33]:
# Nodes

for c in ['ent', 'per', 'pub', 'emp', 'exp']:
    with open('data/' + c + '-posts.json') as f:
        js = json.load(f)
        for e in js[1:]:
            ide = e['ID']
            typ = e['post_type']
            if typ == 'entity':
                name = e['post_title']
                keyword = ''
                typology = ''
                aux = [t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_typology']
                if aux:
                    typology = aux[0]
                    if len(aux)>1:
                        print('ERROR')
                ownership = ''
                aux = [t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_ownership']
                if aux:
                    ownership = aux[0]
                    if len(aux)>1:
                        print('ERROR')
                data = {
                    'id': ide,
                    'type': typ,
                    'name': name,
                    'keyword': keyword,
                    'typology': typology,
                    'ownership': ownership,
                }
            elif typ == 'person':
                name = e['post_title']
                activity = ''
                aux = [t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_activity']
                if aux:
                    activity = aux[0]
                    if len(aux)>1:
                        print('ERROR')
                chronology = ''
                nationality = e['meta']['_cp__peo_country'][0]
                data = {
                    'id': ide,
                    'type': typ,
                    'name': name,
                    'activity': activity
                    'chronology': chronology,
                    'nationality': nationality,
                }
            elif typ == 'book':
                title = e['post_title']
                editorial = ''
                aux = [t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_publisher']
                if aux:
                    editorial = aux[0]
                    if len(aux)>1:
                        print('ERROR')
                date = e['meta']['_cp__boo_publishing_date'][0]
                data = {
                    'id': ide,
                    'type': typ,
                    'title': title,
                    'date': date,
                    'editorial': editorial,
                }
            elif typ == 'company':
                name = e['post_title']
                activity = ''
                nationality = e['meta']['_cp__com_company_headquarter_place'][0].split('; ')[-1]
                data = {
                    'id': ide,
                    'type': typ,
                    'name': name,
                    'activity': activity,
                    'nationality': nationality,
                }
            elif typ == 'exhibition':
                title = e['post_title']
                period = e['meta']['_cp__com_company_headquarter_place'][0].split('; ')[-1]
                artwork_type = ''
                ...
                topic
                typology
                data = {
                    'id': ide,
                    'type': typ,
                    'name': name,
                    'activity': activity,
                    'nationality': nationality,
                }
            
            PROBAR: Para cada registro, mirar cuantos taxonomies tiene del mismo tipo (meter todos los tipos en una
            lista, hacer un Counter y calcular el maximo y ver si es >1). Los q salgan >1 necesitan otra clase.
            
            
            graph.add_node(ide, data)

In [41]:
c=0
for ide, data in graph.nodes_iter(data=True):
    if not data['typology']:
        c+=1
        print(data)

{'typology': '', 'id': 34099, 'ownership': '', 'type': 'entity', 'name': 'FABA. Fundación Almine y Bernard Ruiz-Picasso para el Arte'}
{'typology': '', 'id': 42309, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Adalia'}
{'typology': '', 'id': 42310, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Aguasal'}
{'typology': '', 'id': 42311, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Aguilar de Campos'}
{'typology': '', 'id': 42312, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Alaejos'}
{'typology': '', 'id': 42313, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Alcazarén'}
{'typology': '', 'id': 42314, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Aldea de San Miguel'}
{'typology': '', 'id': 42315, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Aldeamayor de San Martín'}
{'typology': '', 'id': 42316, 'ownership': '', 'type': 'entity', 'name': 'Ayuntamiento de Almenara de Adaja'}
{'typology': ''

In [40]:
c

46